In [14]:

from homeharvest import scrape_property
from datetime import datetime

# Generate filename based on current timestamp
current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"HomeHarvest_{current_timestamp}.csv"

# properties = scrape_property(
#   location="San Diego, CA",
#   listing_type="sold",  # or (for_sale, for_rent, pending)
#   past_days=365*5,  # sold in last 30 days - listed in last 30 days if (for_sale, for_rent)

  # property_type=['single_family','multi_family'],
  # date_from="2023-05-01", # alternative to past_days
  # date_to="2023-05-28",
  # foreclosure=True
  # mls_only=True,  # only fetch MLS listings
#)



In [19]:
from google.cloud import storage

# Init client
client = storage.Client(project='acres-ai')
# Get or create bucket
bucket = client.bucket('scraped-mls')


c:\Users\brand\Documents\projects\acres-ai\venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
### Listed properties pull
# Add ", CA" to each city name
import pandas as pd
list_of_cities = pd.read_csv('../data/city_list.csv')
list_of_cities['City Name'] = list_of_cities['City Name'] + ', CA'

# Initialize empty lists to store results
sold_properties_list = []
listed_properties_list = []

# Iterate through cities
for city in list_of_cities['City Name']:
    print(f"Processing {city}...")

    # Get listed properties    
    try:
        listed = scrape_property(
            location=city,
            listing_type="for_sale",
            past_days=30,
            extra_property_data=True
        )
        if listed is not None:
            listed_properties_list.append(listed)
    except Exception as e:
        print(f"Error scraping listed properties for {city}: {str(e)}")


Processing Alhambra, CA...
Processing Altadena, CA...
Processing Anaheim, CA...
Processing Anaheim Hills, CA...
Processing Arcadia, CA...
Processing Artesia, CA...
Processing Azusa, CA...
Processing Baldwin Park, CA...
Processing Brea, CA...
Processing Buena Park, CA...
Processing Chino, CA...
Processing Chino Hills, CA...
Processing CITY, CA...
Processing Claremont, CA...
Processing Covina, CA...
Processing Diamond Bar, CA...
Processing Downey, CA...
Processing Duarte, CA...
Processing El Monte, CA...
Processing Fountain Valley, CA...
Processing Fullerton, CA...
Processing Garden Grove, CA...
Processing Glendora, CA...
Processing Hacienda Heights, CA...
Processing Hacienda Hts, CA...
Processing La Canada Flintridge, CA...
Processing La Habra, CA...
Processing La Habra Heights, CA...
Processing La Mirada, CA...
Processing La Palma, CA...
Processing La Puente, CA...
Processing La Verne, CA...
Processing Lakewood, CA...
Processing Los Alamitos, CA...
Processing Midway City, CA...
Process

RefreshError: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

In [20]:

listed_properties_df = pd.concat(listed_properties_list) if listed_properties_list else None
# Create folder structure
current_date = datetime.now().strftime("%Y%m%d")
base_path = f"homeharvest/for_sale/{current_date}/"

# Create filename
current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"for_sale_{current_timestamp}.csv"

# Upload to GCS
if listed_properties_df is not None:
    blob = bucket.blob(base_path + filename)
    listed_properties_df.to_csv('temp.csv', index=False)
    blob.upload_from_filename('temp.csv')



In [23]:
listed_properties_df.columns

Index(['property_url', 'property_id', 'listing_id', 'permalink', 'mls',
       'mls_id', 'status', 'mls_status', 'text', 'style', 'formatted_address',
       'full_street_line', 'street', 'unit', 'city', 'state', 'zip_code',
       'beds', 'full_baths', 'half_baths', 'sqft', 'year_built', 'days_on_mls',
       'list_price', 'list_price_min', 'list_price_max', 'list_date',
       'pending_date', 'sold_price', 'last_sold_date', 'last_sold_price',
       'assessed_value', 'estimated_value', 'tax', 'tax_history',
       'new_construction', 'lot_sqft', 'price_per_sqft', 'latitude',
       'longitude', 'neighborhoods', 'county', 'fips_code', 'stories',
       'hoa_fee', 'parking_garage', 'agent_id', 'agent_name', 'agent_email',
       'agent_phones', 'agent_mls_set', 'agent_nrds_id', 'broker_id',
       'broker_name', 'builder_id', 'builder_name', 'office_id',
       'office_mls_set', 'office_name', 'office_email', 'office_phones',
       'nearby_schools', 'primary_photo', 'alt_photos'],
   

In [22]:
listed_properties_df[listed_properties_df['style'] == 'MULTI_FAMILY']


,property_url,property_id,listing_id,permalink,mls,mls_id,status,mls_status,text,style,...,builder_id,builder_name,office_id,office_mls_set,office_name,office_email,office_phones,nearby_schools,primary_photo,alt_photos
3,https://www.realtor.com/realestateandhomes-det...,1104670011,2984546830,713-S-8th-St_Alhambra_CA_91801_M11046-70011,MRCA,OC25164663,FOR_SALE,Active,"Three one-bedroom units, each occupied by long...",MULTI_FAMILY,...,<NA>,<NA>,<NA>,O-MRCA-OC19013,JC Pacific Corp,moonlite513@gmail.com,"[{'number': '6262456226', 'type': 'Office', 'p...",Alhambra Unified School District,<NA>,<NA>
5,https://www.realtor.com/realestateandhomes-det...,1510943927,2984789047,836-S-Sierra-Vista-Ave_Alhambra_CA_91801_M1510...,MRCA,WS25106606,FOR_SALE,Active,This Specious 6 units condo-style like apartme...,MULTI_FAMILY,...,<NA>,<NA>,1426502,O-MRCA-7148,KO TAI REALTY,wangcurt@yahoo.com,"[{'number': '6262858333', 'type': 'Office', 'p...",Alhambra Unified School District,https://ap.rdcpix.com/8199fd164b0c3c34c79e7920...,https://ap.rdcpix.com/8199fd164b0c3c34c79e7920...
19,https://www.realtor.com/realestateandhomes-det...,1188816509,2984564408,1113-W-Hellman-Ave_Alhambra_CA_91803_M11888-16509,MRCA,WS25165217,FOR_SALE,Active,"Priced to sell with instant equity upside, thi...",MULTI_FAMILY,...,<NA>,<NA>,3634329,O-MRCA-21552,Corpen Real Estate Group,<NA>,"[{'number': '4085719626', 'type': 'Office', 'p...",Alhambra Unified School District,https://ap.rdcpix.com/36af9908512ece8bf8462eb7...,https://ap.rdcpix.com/36af9908512ece8bf8462eb7...
21,https://www.realtor.com/realestateandhomes-det...,1385076540,2984761701,601-N-Stoneman-Ave_Alhambra_CA_91801_M13850-76540,MRCA,AR25169737,FOR_SALE,Active,"Offered for sale for the First Time ever, and ...",MULTI_FAMILY,...,<NA>,<NA>,<NA>,O-MRCA-31317,Growth Investment Group Pasade,<NA>,"[{'number': '6265944900', 'type': None, 'prima...",Alhambra Unified School District,https://ap.rdcpix.com/6d766425ab89bf6061144ea4...,https://ap.rdcpix.com/6d766425ab89bf6061144ea4...
38,https://www.realtor.com/realestateandhomes-det...,2400567268,2984514883,920-N-1st-St_Alhambra_CA_91801_M24005-67268,WECA,25567995,FOR_SALE,Active,Situated just two blocks south of the affluent...,MULTI_FAMILY,...,<NA>,<NA>,<NA>,O-WECA-X75151,Marcus & Millichap,<NA>,"[{'number': '(818) 212-2700', 'type': 'Office'...",Alhambra Unified School District,https://ap.rdcpix.com/0753d4b0ca9171eb54310b6e...,https://ap.rdcpix.com/0753d4b0ca9171eb54310b6e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,https://www.realtor.com/realestateandhomes-det...,1125337385,2983931721,13833-Ramona-Dr_Whittier_CA_90605_M11253-37385,MRCA,DW25150369,FOR_SALE,Active,Welcome to 2 gorgeous homes in the Michigan Pa...,MULTI_FAMILY,...,<NA>,<NA>,1036081,O-MRCA-YEXN,"Excellence RE Real Estate, Inc",Broker@ExcellenceRE.com,"[{'number': '(562) 948-4553', 'type': 'Office'...","Whittier Union High School District, Tri-Citie...",https://ap.rdcpix.com/9cb013fc74f21d0013b61493...,https://ap.rdcpix.com/9cb013fc74f21d0013b61493...
81,https://www.realtor.com/realestateandhomes-det...,1604353810,2983751810,10060-Laurel-Ave_Whittier_CA_90605_M16043-53810,MRCA,PV25144648,FOR_SALE,Active,Introducing a investment opportunity at 10060 ...,MULTI_FAMILY,...,<NA>,<NA>,277251,O-MRCA-V8107,Vista Sotheby's International Realty,andy.jolls@vistasir.com,"[{'number': '3105418271', 'type': 'Office', 'p...","Whittier Union High School District, South Whi...",https://ap.rdcpix.com/5bae85821e9e99f675d00629...,https://ap.rdcpix.com/5bae85821e9e99f675d00629...
94,https://www.realtor.com/realestateandhomes-det...,1759068275,2984189779,8366-Sargent-Ave_Whittier_CA_90605_M17590-68275,MRCA,OC25156445,FOR_SALE,Active,"6-Unit Income Property | Private Garages, Rent...",MULTI_FAMILY,...,<NA>,<NA>,2995769,O-MRCA-OC03224,eXp Realty of California Inc.,deborah.penny@exprealty.net,"[{'number': '8885849427', 'type': 'Office', 'p...","Whittier Union High School District, Tri-C

In [ ]:
# Add ", CA" to each city name
list_of_cities = pd.read_csv('../city_list.csv')
list_of_cities['City Name'] = list_of_cities['City Name'] + ', CA'

# Initialize empty lists to store results
sold_properties_list = []
listed_properties_list = []

# Iterate through cities
for city in list_of_cities['City Name']:
    print(f"Processing {city}...")
    
    # Get sold properties
    try:
        sold = scrape_property(
            location=city,
            listing_type="sold", 
            past_days=360,
            extra_property_data=True
        )
        if sold is not None:
            sold_properties_list.append(sold)
    except Exception as e:
        print(f"Error scraping sold properties for {city}: {str(e)}")
        
    # Get listed properties    
    try:
        listed = scrape_property(
            location=city,
            listing_type="for_sale",
            past_days=360,
            extra_property_data=True
        )
        if listed is not None:
            listed_properties_list.append(listed)
    except Exception as e:
        print(f"Error scraping listed properties for {city}: {str(e)}")


Processing Alhambra, CA...
Processing Altadena, CA...
Processing Anaheim, CA...
Processing Anaheim Hills, CA...
Processing Arcadia, CA...
Processing Artesia, CA...
Processing Azusa, CA...
Processing Baldwin Park, CA...
Processing Brea, CA...
Processing Buena Park, CA...
Processing Chino, CA...
Processing Chino Hills, CA...
Processing CITY, CA...
Processing Claremont, CA...
Processing Covina, CA...
Processing Diamond Bar, CA...
Processing Downey, CA...
Processing Duarte, CA...
Processing El Monte, CA...
Processing Fountain Valley, CA...
Processing Fullerton, CA...
Processing Garden Grove, CA...
Processing Glendora, CA...
Processing Hacienda Heights, CA...
Processing Hacienda Hts, CA...
Processing La Canada Flintridge, CA...
Processing La Habra, CA...
Processing La Habra Heights, CA...
Processing La Mirada, CA...
Processing La Palma, CA...
Processing La Puente, CA...
Processing La Verne, CA...
Processing Lakewood, CA...
Processing Los Alamitos, CA...
Processing Midway City, CA...
Process

In [43]:

# Combine results
sold_properties_df = pd.concat(sold_properties_list) if sold_properties_list else None
listed_properties_df = pd.concat(listed_properties_list) if listed_properties_list else None


In [51]:
import pandas as pd

# Upload each dataset to GCS
for properties_df, listing_status in [
    (sold_properties_df, "sold"),
    (listed_properties_df, "for_sale"),
]:
    # Create folder structure
    current_date = datetime.now().strftime("%Y/%m/%d")
    base_path = f"homeharvest/{listing_status}/{current_date}/"
    
    # Create filename
    current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{listing_status}_{current_timestamp}.csv"
    
    # Upload to GCS
    if properties_df is not None:
        blob = bucket.blob(base_path + filename)
        properties_df.to_csv('temp.csv', index=False)
        blob.upload_from_filename('temp.csv')






In [67]:
# Add ", CA" to each city name
list_of_cities = pd.read_csv('../city_list.csv')
list_of_cities['City Name'] = list_of_cities['City Name'] + ', CA'

# Initialize empty lists to store results
sold_properties_list = []
listed_properties_list = []

import json

# Iterate through cities
for city in list_of_cities['City Name']:
    print(f"Processing {city}...")
    
    # Get sold properties
    try:
        sold = scrape_property(
            location=city,
            listing_type="sold", 
            past_days=3600,
            extra_property_data=True,
            return_type="raw"
        )
        if sold is not None:
            sold_properties_list.append(sold)
    except Exception as e:
        print(f"Error scraping sold properties for {city}: {str(e)}")

    # Create folder structure for raw JSON
    current_date = datetime.now().strftime("%Y/%m/%d")
    base_path = f"homeharvest/raw_json/sold_history/"
    
    # Create filename with timestamp
    current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"raw_sold_{city.replace(', ', '_')}_{current_timestamp}.json"
    
    # Upload raw JSON to GCS
    if sold is not None:
        blob = bucket.blob(base_path + filename)
        with open('temp.json', 'w') as f:
            json.dump(sold, f)
        blob.upload_from_filename('temp.json')

        

Processing Alhambra, CA...
Processing Altadena, CA...
Processing Anaheim, CA...
Processing Anaheim Hills, CA...
Processing Arcadia, CA...
Processing Artesia, CA...
Processing Azusa, CA...
Processing Baldwin Park, CA...
Processing Brea, CA...
Processing Buena Park, CA...
Processing Chino, CA...
Error scraping sold properties for Chino, CA: Expecting value: line 1 column 1 (char 0)
Processing Chino Hills, CA...
Processing CITY, CA...
Processing Claremont, CA...
Processing Covina, CA...
Processing Diamond Bar, CA...
Processing Downey, CA...
Processing Duarte, CA...
Processing El Monte, CA...
Processing Fountain Valley, CA...
Processing Fullerton, CA...
Processing Garden Grove, CA...
Processing Glendora, CA...
Processing Hacienda Heights, CA...
Processing Hacienda Hts, CA...
Processing La Canada Flintridge, CA...
Processing La Habra, CA...
Processing La Habra Heights, CA...
Processing La Mirada, CA...
Processing La Palma, CA...
Processing La Puente, CA...
Processing La Verne, CA...
Process

In [ ]:
from google.cloud import storage
import pandas as pd
import os
# Init client
client = storage.Client(project='acres-ai')
# Get or create bucket
bucket = client.bucket('scraped-mls')


NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/scraped-mls/o/homeharvest%2Fprocessed%2Ffor_sale%2Flatest.csv?alt=media: No such object: scraped-mls/homeharvest/processed/for_sale/latest.csv: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [5]:

# Read the for sale CSV from Google Cloud Storage
for_sale_blob = bucket.blob('homeharvest/for_sale/2025/07/25/for_sale_20250725_012150.csv')

# Download to a temporary file
for_sale_blob.download_to_filename('temp_for_sale.csv')

# Read into pandas
for_sale_df = pd.read_csv('temp_for_sale.csv')

# Clean up temp file
os.remove('temp_for_sale.csv')


In [12]:
# Read the for sale CSV from Google Cloud Storage
for_sale_blob = bucket.blob('homeharvest/for_sale/2025/07/25/for_sale_20250725_012150.csv')

# Download to a temporary file
for_sale_blob.download_to_filename('temp_for_sale.csv')

# Read into pandas and remove duplicates
for_sale_df = pd.read_csv('temp_for_sale.csv')
for_sale_df = for_sale_df.drop_duplicates(subset=['listing_id'], keep='first')

# Write back to same location in GCS
for_sale_df.to_csv('temp_for_sale.csv', index=False)
for_sale_blob.upload_from_filename('temp_for_sale.csv')

# Clean up temp file
os.remove('temp_for_sale.csv')


In [11]:
# Check for duplicate listing_ids
duplicate_listings = for_sale_df[for_sale_df.duplicated(subset=['listing_id'], keep=False)]
print(f"\nFound {len(duplicate_listings)} duplicate listings:")
print(duplicate_listings.sort_values('listing_id'))


Found 1667 duplicate listings:
                                           property_url  property_id  \
1477  https://www.realtor.com/realestateandhomes-det...   2516675166   
924   https://www.realtor.com/realestateandhomes-det...   2516675166   
7737  https://www.realtor.com/realestateandhomes-det...   2281903671   
7264  https://www.realtor.com/realestateandhomes-det...   2281903671   
1301  https://www.realtor.com/realestateandhomes-det...   9781716494   
...                                                 ...          ...   
1714  https://www.realtor.com/realestateandhomes-det...   1822754793   
923   https://www.realtor.com/realestateandhomes-det...   9486200116   
1476  https://www.realtor.com/realestateandhomes-det...   9486200116   
963   https://www.realtor.com/realestateandhomes-det...   2700206784   
1503  https://www.realtor.com/realestateandhomes-det...   2700206784   

      listing_id                                          permalink   mls  \
1477  2963523943       214

In [24]:
from redfin_scraper import RedfinScraper
scraper = RedfinScraper()




In [25]:
scraper.setup(r'C:\Users\brand\Documents\projects\acres-ai\zip_code_database.csv', multiprocessing=True)

In [29]:
# Define the parameters for the scrape
city_states = ['Alhambra, CA']
#zip_codes = ['91803']
sold = False
sale_period = 'last_30_days'
lat_tuner = 1.5
lon_tuner = 1.5

# Call the scrape method with the defined parameters
scraper.scrape(city_states=city_states, sold=sold, sale_period=sale_period, lat_tuner=lat_tuner, lon_tuner=lon_tuner)


In [31]:
scraper.get_data('D002')



In [35]:
scraper.data

{'D001': None, 'D002': None}